#Testing our models
This file helps in creates our 

In [1]:
#
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/COLAB/DL_3D_MIA_project/metrics_acdc.py /content

!pip install SimpleITK
!pip install nibabel
!pip install skimage
!pip install medpy
!pip install monai


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 26 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement skimage (from versions: 0.0)
ERROR: No matching distribution found for skimage
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 28.8 MB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754481 sha256=b202219363d00d04e97389d20fe32ae6b22ed49c3b95ac9a3369673abac53823
  Stored in directory: /root/.cache/pip/wheels/b0/57/3a/da1183f22a6afb42e11138daa6a759de233fd977a984333602
Successfully built medpy
Looking in indexes: https://p

In [2]:
#Import all the required files
import SimpleITK as sitk
import nibabel as nib
import monai

import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import glob

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from monai.transforms import LoadImage, SaveImage, Transform
from metrics_acdc import load_nii
from torch.nn.functional import pad
from monai.transforms.utils_pytorch_numpy_unification import maximum
from numpy import floor_divide
from monai.transforms import SaveImage, Spacing
from monai.data import create_test_image_3d, list_data_collate, decollate_batch

systolic_mode = 'ES'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# build dict of ACDC data

model_path = '/content/drive/My Drive/COLAB/DL_3D_MIA_project/models/'
data_path = r'/content/drive/My Drive/COLAB/DL_3D_MIA_project/data/ACDC/testing'

print(f'The used device is {device}')

The used device is cpu


# Construct the dictionary of file paths

In [3]:
# split of dataset in train and val must be based on patients not on image slices
train_patient, validation_patient = train_test_split(range(1,101), train_size=0.8, random_state=42)
test_patient = range(101, 151)
print(train_patient)
print(validation_patient)
print(test_patient)

def build_dict_acdc(data_path, prediction_path='', mode='train'):
    """
    This function returns a list of dictionaries, each dictionary containing the keys 'img' and 'mask' 
    that returns the path to the corresponding image.
    
    Args:
        data_path (str): path to the root folder of the data set.
        mode (str): subset used. Must correspond to 'train', 'val' or 'test'.
        
    Returns:
        (List[Dict[str, str]]) list of the dictionaries containing the paths of images and masks.
    """
    # test if mode is correct
    if mode not in ["train", "val", "test"]:
        raise ValueError(f"Please choose a mode in ['train', 'val', 'test']. Current mode is {mode}.")
    
    # define empty dictionary
    dicts = []
    if mode == "test":
        for patient_nb in list(test_patient):
            # set patient_path
            patient_path = os.path.join(data_path, 'patient'+'{0:03}'.format(patient_nb))

            # extract ED and ES frame number from patient info
            patient_info = os.path.join(patient_path, 'Info.cfg')
            file = open(patient_info, 'r').readlines()
            ED_frame = int(file[0].split(':')[1])
            ES_frame = int(file[1].split(':')[1])

            # set image and mask paths
            image_path = []
            image_path.append(os.path.join(patient_path, 'patient'+'{0:03}'.format(patient_nb)+'_frame'+'{0:02}'.format(ED_frame)+'.nii.gz'))
            image_path.append(os.path.join(patient_path, 'patient'+'{0:03}'.format(patient_nb)+'_frame'+'{0:02}'.format(ES_frame)+'.nii.gz'))
           
            mask_path = []
            dicts.append({'img': image_path, 'mask': mask_path})
    else:
        target = train_patient if mode == "train" else validation_patient 
        # make a corresponding list for all the mask files
        for patient_nb in target:
            # set patient_path
            patient_path = os.path.join(data_path, 'patient'+'{0:03}'.format(patient_nb))

            # extract ED and ES frame number from patient info
            patient_info = os.path.join(patient_path, 'Info.cfg')
            file = open(patient_info, 'r').readlines()
            ED_frame = int(file[0].split(':')[1])
            ES_frame = int(file[1].split(':')[1])

            # set image and mask paths
            image_path = []
            image_path.append(os.path.join(patient_path, 'patient'+'{0:03}'.format(patient_nb)+'_frame'+'{0:02}'.format(ED_frame)+'.nii.gz'))
            image_path.append(os.path.join(patient_path, 'patient'+'{0:03}'.format(patient_nb)+'_frame'+'{0:02}'.format(ES_frame)+'.nii.gz'))

            mask_path = []
            mask_path.append(os.path.join(patient_path, 'patient'+'{0:03}'.format(patient_nb)+'_frame'+'{0:02}'.format(ED_frame)+'_gt.nii.gz'))
            mask_path.append(os.path.join(patient_path, 'patient'+'{0:03}'.format(patient_nb)+'_frame'+'{0:02}'.format(ES_frame)+'_gt.nii.gz'))
            
            prediction_preprocessing_path = []
            
            if mode != "test":
                prediction_preprocessing_path.append(os.path.join(prediction_path,'predicted_segmentation_patient'+str(patient_nb)+'.npy'))
                prediction_preprocessing_path.append(os.path.join(prediction_path,'predicted_segmentation_patient'+str(patient_nb)+'.npy'))

            dicts.append({'img': image_path, 'mask': mask_path, 'prediction_preprocessing': prediction_preprocessing_path})
    return dicts

[56, 89, 27, 43, 70, 16, 41, 97, 10, 73, 12, 48, 86, 29, 94, 6, 67, 66, 36, 17, 50, 35, 8, 96, 28, 20, 82, 26, 63, 14, 25, 4, 18, 39, 9, 79, 7, 65, 37, 90, 57, 100, 55, 44, 51, 68, 47, 69, 62, 98, 80, 42, 59, 49, 99, 58, 76, 33, 95, 60, 64, 85, 38, 30, 2, 53, 22, 3, 24, 88, 92, 75, 87, 83, 21, 61, 72, 15, 93, 52]
[84, 54, 71, 46, 45, 40, 23, 81, 11, 1, 19, 31, 74, 34, 91, 5, 77, 78, 13, 32]
range(101, 151)


In [4]:
test_dict_list = build_dict_acdc(data_path, mode='test')
print('Size of test dataset: ', len(test_dict_list))

Size of test dataset:  50


# Define a function that loads the middle slices (ED only) of the ACDC dataset

In [5]:
class LoadACDCData(monai.transforms.Transform):
    """
    This custom Monai transform loads the data from the acdc segmentation dataset.
    Defining a custom transform is simple; just overwrite the __init__ function and __call__ function.
    """
    def __init__(self, keys=None):
        pass

    def __call__(self, sample):
        q = 0 if systolic_mode == "ED" else 1
        image_q = load_nii(sample['img'][q])
        image = image_q[0][:,:,4]
        _, image_q_aff, image_q_header = load_nii(sample['img'][q])
        # remove third row and column to get affine matrix corresponding to 2D image
        image_q_aff = np.delete(image_q_aff, 2, 0)
        image_q_aff = np.delete(image_q_aff, 2, 1)
        # set pixel dimensions in image_ED_aff
        image_q_aff[0,0] = -image_q_header['pixdim'][1]
        image_q_aff[1,1] = -image_q_header['pixdim'][2]

        return {'img': image, 'img_meta_dict': {'affine': image_q_aff, 'pixdim': image_q_header['pixdim']}}

In [6]:
# create dataset with cache mechanism that can load data and cache deterministic transforms’ result during training
test_dataset = monai.data.CacheDataset(test_dict_list, transform=LoadACDCData())

Loading dataset: 100%|██████████| 50/50 [00:18<00:00,  2.68it/s]


# Define the transforms that should be applied on the test data
We add a channel in all of our data and perform intensitiy scaling with a lower bound of 5 and upper bound of 95.
Furthermore, all our images are sampled to the same spatial resolution of 1.5mm^2

In [7]:
test_transform = monai.transforms.Compose(
    [
        LoadACDCData(),
        monai.transforms.AddChanneld(keys=['img']),
        monai.transforms.ScaleIntensityRangePercentilesd(keys=['img'], lower=5, upper=95, b_min=0, b_max=1),
        monai.transforms.Spacingd(keys=['img'], 
                                  pixdim=(1.5, 1.5), 
                                  mode=("bilinear"))
    ]
)

In [8]:
test_dataset_transformed = monai.data.CacheDataset(test_dict_list, transform=test_transform)
test_dataloader = monai.data.DataLoader(test_dataset_transformed, batch_size=16, shuffle=True)
print('Length of test_dataloader: ', len(test_dataloader))


Loading dataset: 100%|██████████| 50/50 [00:04<00:00, 11.65it/s]

Length of test_dataloader:  4


## Calculate ROI for test set

In [9]:
#This model is the pretrained ROI-Extracting model
model = monai.networks.nets.UNet(
    dimensions=2,
    in_channels=1,
    out_channels=1,
    channels = (16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

filename = '20220615_1137_model_preprocess.pt'
model.load_state_dict(torch.load(model_path+filename))

RuntimeError: ignored

In [ ]:
#Several functions are defined here

### Returns the largest connected component of a tensor
def getLargestCC(output):
    labels = skimage.measure.label(output)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC

### Using a sliding window inference, the test images are masked using the ROI-extracting UNet
def visual_evaluation(sample, model):
    """
    Allow the visual inspection the result of one sample by plotting 
    the X-ray image, the ground truth (green) and the segmentation map produced by the network (red).
    
    Args:
        sample (Dict[str, torch.Tensor]): sample composed of an X-ray ('img') and a mask ('mask').
        model (torch.nn.Module): trained model to evaluate.
    """
    model.eval()
    inferer = monai.inferers.SlidingWindowInferer(roi_size=[128,128], overlap=0.25, mode='gaussian')
    discrete_transform = monai.transforms.AsDiscrete(logit_thresh=0.5, threshold_values=True)
    Sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        output = Sigmoid(inferer(sample['img'].to(device), network=model).cpu()).squeeze()
        output = discrete_transform(output)
        print(np.unique(output))
    
    output = getLargestCC(output)
    output = output.astype(int)
    ###IMPORTANT
    ###Calculate the center coordinates of the ROI by using the center of mass of the largest connected component.
    center_coordinates = scipy.ndimage.measurements.center_of_mass(output)
    center_coordinates = np.round(center_coordinates)   # round center_coordinates to get int values
    print('center: ', center_coordinates)

    plt.figure()
    plt.imshow(output)
    plt.scatter(center_coordinates[1], center_coordinates[0], color='r')
    plt.show()
    
    fig, ax = plt.subplots(1,2, figsize = [12, 10])
    
    # Plot X-ray image
    ax[0].imshow(sample["img"].squeeze(), 'gray')
    # Plot output
    overlay_output = np.ma.masked_where(output < 0.1, output >0.99)
    ax[1].imshow(sample['img'].squeeze(), 'gray')
    ax[1].imshow(overlay_output, 'Greens', alpha = 0.7, clim=[0,1])
    ax[1].set_title('Prediction')
    plt.show()

    return output, center_coordinates

In [ ]:
test_dict_list_evaluation = build_dict_acdc(data_path, mode='test')
test_transform_evalation = monai.transforms.Compose([
        LoadACDCData(),
        monai.transforms.AddChanneld(keys=['img']),
        monai.transforms.ScaleIntensityRangePercentilesd(keys=['img'], lower=5, upper=95, b_min=0, b_max=1),
        monai.transforms.Spacingd(keys=['img'], 
                                  pixdim=(1.5,1.5), 
                                  mode=['bilinear']),
    ]
)
test_dataset_evaluation = monai.data.CacheDataset(test_dict_list_evaluation, transform=test_transform_evalation)
test_dataloader_evaluation = monai.data.DataLoader(test_dataset_evaluation, batch_size=1)


In [ ]:
#Print out all the coordinates of the ROI-crop and store these in lists
output_test = []
center_coordinates_test = []
for i, sample in enumerate(test_dataloader_evaluation):
    print('patient number: ', test_patient[i])
    output_sample, center_sample = visual_evaluation(sample, model)
    output_test.append(output_sample)
    center_coordinates_test.append(center_sample)


#Segmentation
After having have received the coordinates for each of the Test patients heart location, we reload the dataset and load the model we want to test (Either UNET or DCNN for ES or ED).

NB: Most explanations of the code are in the code fragments

## Reload the dataset 
After we haveobtained the approximate coordinates for the heart, we will have to reload the dataset to include all slices. Then we can transform them with the SpatialCrop at the previously determined coordinates

In [ ]:
class LoadACDCData(monai.transforms.Transform):
    """
    This custom Monai transform loads the data from the acdc segmentation dataset.
    Defining a custom transform is simple; just overwrite the __init__ function and __call__ function.
    """
    def __init__(self, keys=None):
        pass

    def __call__(self, sample):
        q = 0 if systolic_mode == "ED" else 1
        image_q = load_nii(sample['img'][q])
        image = image_q[0][:,:,4]
        _, image_q_aff, image_q_header = load_nii(sample['img'][q])
        # remove third row and column to get affine matrix corresponding to 2D image
        image_q_aff = np.delete(image_q_aff, 2, 0)
        image_q_aff = np.delete(image_q_aff, 2, 1)
        # set pixel dimensions in image_ED_aff
        image_q_aff[0,0] = -image_q_header['pixdim'][1]
        image_q_aff[1,1] = -image_q_header['pixdim'][2]

        return {'img': image, 'img_meta_dict': {'affine': image_q_aff, 'pixdim': image_q_header['pixdim']}}

In [ ]:
test_transform = monai.transforms.Compose(
    [
        LoadACDCData(),
        monai.transforms.AddChanneld(keys=['img']),
        monai.transforms.ScaleIntensityRangePercentilesd(keys=['img'], lower=5, upper=95, b_min=0, b_max=1),
        monai.transforms.Spacingd(keys=['img'], 
                                  pixdim=(1.5, 1.5), 
                                  mode=("bilinear")),
    ]
)

test_dataset_transformed = monai.data.CacheDataset(test_dict_list, transform=test_transform)

In this block we store original data from the original images to recreate the images after we have transformed them and savely output them backtowards the NifTi format for evaluation on the ACDC website.

In [ ]:

i=0
original_3D_shapes = {}
original_3D_pixdims = {}
original_3D_affine = {}
for sample in test_dataset_transformed:
    patient_nb = test_patient[i]
    image = sample["img"]
    pixdim = sample["img_meta_dict"]['pixdim']
    affine = sample["img_meta_dict"]['affine']
    original_3D_shapes[patient_nb] = list(image.shape)
    original_3D_pixdims[patient_nb] = pixdim
    original_3D_affine[patient_nb] = affine
    print(i, ": Image size: ", image.shape, "Pixel Dimensions: ", pixdim
          # , '\n Affine: \n', affine
          )
    i=i+1
print(original_3D_shapes)

The below function takes as input the test patient (3D)dataset and returns the slices cropped at the ROI coordinates.

In [ ]:
# change 3D dataset to 2D dataset
def change_dataset_from_3D_to_2D(dataset_3D, center_coordinates):
  samples = []
  for j, sample_3D in enumerate(dataset_3D):
    print(j)
    image_3D = sample_3D['img']
    print(image_3D.shape)
    for i in range(image_3D.shape[3]):
      print(center_coordinates)
      center_coordinates_i = center_coordinates[j]
      crop_transform = monai.transforms.SpatialCrop(roi_center=(int(center_coordinates_i[0]), int(center_coordinates_i[1])),
                                                    roi_size=(128, 128))

      sample_2D = {}
      sample_2D['img'] = crop_transform(image_3D[:,:,:,i])
      sample_2D['img_meta_dict'] = sample_3D['img_meta_dict']
      samples.append(sample_2D)

  return samples

In [ ]:
test_dataset_2D = change_dataset_from_3D_to_2D(test_dataset_transformed, center_coordinates_test)
print('Size of 2D testing dataset: ', len(test_dataset_2D))

i=0
for sample in test_dataset_2D:
    i=i+1
    image = sample["img"]
    meta = sample["img_meta_dict"]
    print(i, ": Image size: ", image.shape, "| meta:", meta)

## Load the segmentation model
The previously pre-trained Dilated CNN will be reloaded to give predictions on the testing images in 2D. After predictions, the slices will be stacked and transformed back to their original patients MRI 3D Image.

NB: To test a different model, you must change the filename in the below code block.

In [ ]:
import torch
import torch.nn as nn

from typing import Any, Optional, Sequence, Tuple, Union

from monai.networks.blocks.convolutions import Convolution, ResidualUnit, ADN
from monai.networks.layers.factories import Act, Norm
from monai.networks.layers.simplelayers import SkipConnection
from monai.utils import alias, deprecated_arg

class CNN(nn.Module):
    
    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[Sequence[int], int] = 3,
    ) -> None:

        super().__init__()

        self.conv1 = Convolution(
          spatial_dims=2,
          kernel_size=9,
          in_channels=1,
          out_channels=16,
          adn_ordering="NDA",
          act="relu",
          norm="batch",
          dilation=1,
        )
        self.conv2 = Convolution(
          spatial_dims=2,
          kernel_size=7,
          in_channels=16,
          out_channels=64,
          adn_ordering="NDA",
          act="relu",
          norm="batch",
          dilation=2,
        )
        self.conv3 = Convolution(
          spatial_dims=2,
          kernel_size=5,
          in_channels=64,
          out_channels=64,
          adn_ordering="NDA",
          act="relu",
          norm="batch",
          dilation=4,
        )
        self.conv4 = Convolution(
          spatial_dims=2,
          kernel_size=3,
          in_channels=64,
          out_channels=64,
          adn_ordering="NDA",
          act="relu",
          norm="batch",
          dilation=6,
        )
        self.conv5 = Convolution(
          spatial_dims=2,
          kernel_size=3,
          in_channels=64,
          out_channels=4,
          adn_ordering="NDA",
          act=None,
          norm="batch",
          dilation=8
        )
        self.final = torch.nn.Softmax()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #x = self.model(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.final(x)

        return x

model = CNN(
    spatial_dims=2,
    in_channels=1,
    out_channels=4,
).to(device)

#TODO: Change this to the desired model
filename = '20220701_1526_model_main_DCNN_ED.pt'
model.load_state_dict(torch.load(model_path+filename))

## Inference

In [ ]:
test_dataloader = monai.data.DataLoader(test_dataset_2D, batch_size=1)
print(len(test_dataloader))

In [ ]:
import time
model.eval()
results = []
for i, test_data in enumerate(test_dataloader):
    test_data = test_data["img"].to(device)
    print(test_data.shape)
    inferer = monai.inferers.SimpleInferer()
    discrete_transform = monai.transforms.AsDiscrete(logit_thresh=0.5, threshold_values=True)
    with torch.no_grad():
        tic = time.perf_counter()
      
     
        output = inferer(inputs=test_data, network=model).cpu().squeeze()
        output = discrete_transform(output)
        toc = time.perf_counter()
        print(f"Inference in {toc - tic:0.4f} seconds")
    results.append(output)
  

## Save predictions
First we transform the predicted output of size 128x128 back into its original shape. Then we assign each patient their individual slices and resample our spatial resolution back towards the original spatial resolution provided in the meta data of the file. We must save the predictions to nii.gz format in order to compute the metrics on the ACDC Website.

In [ ]:
nb_slices_per_patient_test=[]

i=0
for sample in test_dataset_transformed:
    i=i+1
    image = sample["img"]
    nb_slices = sample['img'].shape[3]
    nb_slices_per_patient_test.append(nb_slices)
    #print(i, ": Image size: ", image.shape)

print(nb_slices_per_patient_test)


In [ ]:
#Stack all the slices per patient to recreate the original models
output_test_3D = []
#mask_test_3D = []
index = 0
for i, patient_number in enumerate(test_patient):
  output_test_3D_i = torch.tensor(np.zeros([4,128,128,nb_slices_per_patient_test[i]]))
  for slice_i in range(nb_slices_per_patient_test[i]):
    output_test_3D_i[:,:,:,slice_i] = results[index]
    index=index+1
  output_test_3D.append(output_test_3D_i)

print(len(output_test_3D))


In [ ]:
#Retrieve all metadata in a list
meta = []
for test in test_dataset_transformed:
    meta.append(test['img_meta_dict'])
#print(meta)

In [ ]:
segmented_patients = {}
for i in range(len(test_patient)):
    
    meta_data = meta[i]
    information = {}
    information['img'] = output_test_3D[i]
    information['img_meta_dict'] = meta_data
    segmented_patients[test_patient[i]] = information
    
print(segmented_patients[101]['img'].shape)
#All patients now have the segmented images and their original affine matrix after transformation
# #For each slice in the predicted segmentation create an empty multichannel tensor
# shape = (4, (original_shape[1]), (original_shape[2]))
# seg = torch.zeros(shape)
# #print(seg.shape)

In [ ]:
#Recreates original shapes by padding the missing edges from the ROI coordinates
def recreate_original_matrix(original_shape, image, coordinates, roi_size=128):
    output = []
    channel = 0
    for i in image:
        padded_slices = []
        for j in range(image.shape[3]):
            to_pad = image[channel, :, :, j]

            roi_start = coordinates - floor_divide(roi_size, 2)
            roi_end = maximum(roi_start + roi_size, roi_start)
            
            x, y = original_shape[1], original_shape[2]
            pad_directions = (int(roi_start[1]), int(y - roi_end[1]), int(roi_start[0]), int(x - roi_end[0]))
            #Pad background with missing 1s and all other classes with missing 0s
            mode = 1 if channel == 0 else 0
            padded_image = pad(to_pad, pad_directions, 'constant', mode)
            #print("New image shape:", padded_image.shape)
            padded_slices.append(padded_image) 
          
        padded_slices = torch.stack(padded_slices, dim=2)
        output.append(padded_slices)
        channel += 1
    output = torch.stack(output, dim=0)      
        
    print(output.shape)
    return output
 

In [ ]:
os.chdir('/content/drive/My Drive/COLAB/DL_3D_MIA_project/masks')
saver = SaveImage(output_dir='/content/drive/My Drive/COLAB/DL_3D_MIA_project/masks', output_ext=".nii.gz", output_postfix='ED', separate_folder=False)

index = 0
for nb, value in segmented_patients.items():
    original_shape = original_3D_shapes[nb]
    or_pixdim = original_3D_pixdims[nb]
    or_affine = original_3D_affine[nb] 
    image = value['img']
    meta = value['img_meta_dict']
    #get the coordinates for this patient
    coordinates = center_coordinates_test[index]
    #Reformat to original spacing
    recreated_image = recreate_original_matrix(original_shape, image, coordinates)
    spacer = Spacing(pixdim=(or_pixdim[1], or_pixdim[2]), mode="bilinear")
    recreated_image, new_affine, new_pixdim = spacer(recreated_image, affine=or_affine)
    #save to file
    fname = "patient" + str(nb)
    meta['filename_or_obj'] = fname
    fname = decollate_batch(meta['filename_or_obj'])

    # plot predicted segmentation once
    if index == 0:
        for j in range(10):
          # best prediction
          fig = plt.figure(figsize=(24, 24))

          plt.subplot(2, 4, 1)
          plt.imshow(np.transpose(recreated_image[1,:,:,j]), cmap='Reds')
          plt.imshow(np.transpose(recreated_image[2,:,:,j]), cmap='Greens', alpha = 0.4, clim=[0,1])
          plt.imshow(np.transpose(recreated_image[3,:,:,j]), cmap='Blues', alpha = 0.4, clim=[0,1])
          plt.subplot(2, 4, 2)
          plt.imshow(np.transpose(recreated_image[1,:,:,j]), cmap='Reds')
          # plt.imshow(np.transpose(mask_val_3D[12][1,:,:,3]), 'Reds', alpha = 0.2, clim=[0,1])
          plt.subplot(2, 4, 3)
          plt.imshow(np.transpose(recreated_image[2,:,:,j]), cmap='Greens')
          plt.subplot(2, 4, 4)
          plt.imshow(np.transpose(recreated_image[3,:,:,j]), cmap='Blues')

    saver(recreated_image, meta)
    index += 1

Open the newly saved files and plot the image masks with the original image


In [ ]:
def plot_pred(or_image, seg):
    # plot predicted segmentation
    for i in range(or_image.shape[2]):
        plt.figure()
        plt.imshow(or_image[:,:,i].transpose(), 'gray')
        mask = seg[:,:,i,:]
        overlay_mask = np.ma.masked_where(mask == 0, mask == 1)
        plt.imshow(overlay_mask[:,:,1].transpose(), 'Reds', alpha=0.7, clim=[0,1], interpolation='nearest')
        plt.imshow(overlay_mask[:,:,2].transpose(), 'Greens', alpha=0.7, clim=[0,1], interpolation='nearest')
        plt.imshow(overlay_mask[:,:,3].transpose(), 'Blues', alpha=0.7, clim=[0,1], interpolation='nearest')
        plt.show()

In [ ]:
segmentation_path = r'/content/drive/My Drive/COLAB/DL_3D_MIA_project/masks/'
sample = segmentation_path + 'patient101_ED.nii.gz'
original = data_path + "/patient101/patient101_frame01.nii.gz"

or_image, or_image_affine, or_image_header = load_nii(original)
image, image_affine, image_header = load_nii(sample)

print(or_image.shape)
print(image.shape)

plot_pred(or_image, image)

